[Reference](https://medium.com/@duzieblaise/detecting-maize-crop-diseases-with-yolov8-an-ai-project-in-precision-agriculture-e45c2b06fa68)

# Data Collection and Annotation
- [Link](https://roboflow.com/)

In [1]:
#Phase 1
# Install required libraries
!pip install ultralytics

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# import yolo
from ultralytics import YOLO
model = YOLO("yolov8n-seg.pt")

# setup paths
import os
drive_project_path = '/content/drive/My Drive/maize-disease-detection/'
os.makedirs(drive_project_path, exist_ok=True)

# unzip dataset to drive
!unzip /content/maize-disease-dataset.zip -d {drive_project_path}

# List files in dataset
!ls {drive_project_path}

# Update dataset path to drive location
results = model.train(
data=f"{drive_project_path}data.yaml",
epochs=60, # model trains on the whole dataset 60 times
imgsz=640,
batch=16,
project=drive_project_path  # Save training results to Drive
)

# Phase 2 - To run the inference on photos of Diseased maize
import glob
import os
from IPython.display import Image, display
from ultralytics import YOLO
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set up project paths
drive_project_path = '/content/drive/My Drive/maize-disease-detection/'
os.makedirs(drive_project_path, exist_ok=True)

# Load the model from Drive
model = YOLO(f"{drive_project_path}runs/segment/train/weights/best.pt")

# Run inference on test images
results = model.predict(
source=f"{drive_project_path}maize-disease-dataset/test/images",
save=True,
project=drive_project_path
)

# Display predicted images
predicted_images_dir = f'{drive_project_path}runs/segment/predict/'
for image_path in glob.glob(predicted_images_dir + '*.jpg')[:10]:
display(Image(filename=image_path, height=400))
print('\n')

# Export model to onnx format in Drive
onnx_model_path = model.export(
format='onnx',
project=drive_project_path
)

# Download ONNX model
print("\nDownload the exported ONNX model:")
files.download(onnx_model_path)


# Phase 3 - To run the inference on Already recorded video of Diseased maize
from IPython.display import Video, display
from google.colab import drive, files
from ultralytics import YOLO
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set project paths
drive_project_path = '/content/drive/My Drive/maize-disease-detection/'
os.makedirs(drive_project_path, exist_ok=True)

# Upload Videos
print("Upload your video file:")
uploaded = files.upload()

 # Get the uploaded video filename
video_filename = list(uploaded.keys())[0]
video_path = f'/content/{video_filename}'

# Load YOLO model
model = YOLO(f"{drive_project_path}runs/segment/train/weights/best.pt")

# Run inference on the uploaded video
print("Running inference on the video...")
results = model.predict(
source=video_path,
save=True,
stream=True,
project=drive_project_path
)

# Path to the processed video
processed_video_path = f'{drive_project_path}{os.path.splitext(video_filename)[0]}_processed.mp4'

# Display the processed video in Colab
print("\nProcessed Video:")
display(Video(processed_video_path, width=800, embed=True))

# Option to download the processed video
print("\nDownload the processed video:")
files.download(processed_video_path)

# The original intention was to upload the code to a repository
# but due to safety concerns I had to refrain from this.

# The benefits of using YOLO are numerous as it handles all the complex algorithms behind this and provides you
# with methods to make the training code way simpler.